In [1]:
import pandas as pd
import numpy as np 
import scipy
import math 
from scipy import stats
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal 

iv = pd.read_excel("Impvols_SPX_AMZN.xlsx", header = 1)
spx_strikes = iv.spx_strikes.dropna().to_numpy()
am_strikes = iv.amzn_strikes.dropna().to_numpy()
am_iv = iv["amzn implied vols"].dropna().to_numpy()
spx_iv = iv["spx implied vols"].dropna().to_numpy()

In [2]:
def black_scholes_call(t, T, s, k, r, sigma, q = 0):
    d1 = ( np.log(s/k) + (r - q + 0.5*sigma**2)*(T-t) )/(sigma * np.sqrt(T-t))
    d2 = d1 - sigma * np.sqrt(T-t)
    price = s * np.exp(-q *(T-t))*scipy.stats.norm.cdf(d1) - k*np.exp(-r *(T-t))*scipy.stats.norm.cdf(d2)
    return price

t = 0
r = 2.4/100
T = 108/365
am_q = 1.9/100
spx_q = 1.8/100
am_s0 = 1971
spx_s0 = 2921

#undiscounted price 
am_price = np.exp(r*T) * black_scholes_call(t, T, am_s0, am_strikes, r, am_iv, am_q)
spx_price = np.exp(r*T) * black_scholes_call(t, T, spx_s0, spx_strikes, r, spx_iv, spx_q)

In [14]:
np.random.seed(100)

def implied_cdf(strikes, call_price):
    c1, k1 = call_price[1:], strikes[1:]
    c2, k2 = call_price[0:-1], strikes[0:-1]
    return 1 + (c1 - c2)/(k1 - k2)    #this is the cdf calculated in k1 

#therefore we can easily connect each element of the cdf with the corresponding k
#in fact the first element of the cdf, is the cdf calculated in the first element of the strike array 

am_cdf = implied_cdf(am_strikes, am_price)
spx_cdf = implied_cdf(spx_strikes, spx_price)

def get_inverse(strike, cdf, level):
    index = np.argmin(np.power(cdf - level, 2))
    return strike[index]

rho = 0.5
n_simu = 10000
simu = np.random.multivariate_normal([0,0], np.array([[1,0.5], [0.5, 1]]),  n_simu)    
simu_1, simu_2 = simu[:,0], simu[:,1]

q1, q2 = scipy.stats.norm.cdf(simu_1), scipy.stats.norm.cdf(simu_2)

am_val = np.array([get_inverse(am_strikes, am_cdf, q) for q in q1])
spx_val = np.array([get_inverse(spx_strikes, spx_cdf, q) for q in q2])

payoff = (spx_val/spx_s0 - am_val/am_s0)
payoff = (payoff>0)*payoff

price = np.exp(-r*T)*np.mean(payoff)
print(price)

0.057632871818947086


In [15]:
#additional method from numpy 
#we first calculate the variance of each payoff then we calculate the variance of the average of payoff 
variance = np.var(np.exp(-r*T)*payoff)
avg_variance = 1/n_simu * variance
std = np.sqrt(avg_variance)
print(std)

0.0008872388106103867


0.0036101083032490976